In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
from datetime import datetime, timezone
import time

import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import json

from binance.client import Client

from datetime import datetime, timezone, timedelta

Profit (GBP) = Pips x Exposure / GBPUSD = 0.002 x 2,000 / 1.5185 = £2.63



In [13]:
from oandapyV20 import API
import oandapyV20.endpoints.instruments as instruments

from os.path import expanduser

oanda_headers = json.loads(open("{0}/mrpozzi_oanda_token.json".format(expanduser("~"))).read())
oanda_demo_headers = json.loads(open("{0}/mrpozzi_oanda_practice_token.json".format(expanduser("~"))).read())

TOKEN = oanda_headers['token']
DEMO_TOKEN = oanda_demo_headers['token']


In [14]:
import sys
sys.path.append("{0}/repos/imprint/".format(expanduser("~")))
from trading.utils.data_processing import process_oanda_candle
from backtest import BATCH_SIZE

# Generator

In [19]:
from generators.oanda_candles_generator import OandaCandlesGenerator

DEMO = True
if DEMO:
    credentials = "{0}/mrpozzi_oanda_practice_token.json".format(expanduser("~"))
else:
    credentials = "{0}/mrpozzi_oanda_token.json".format(expanduser("~"))

candles_generator = OandaCandlesGenerator(
    credentials=credentials,
    symbol="EUR_USD",
    demo_mode=DEMO
)

DEBUG:imprint_trading_bot:processing mode: on_change


In [20]:
candles_generator.initial_batch(False)

,close_time,open,high,low,close,ohlc4,volume,price,complete
0,2021-08-05 00:00:00+00:00,1.18383,1.18394,1.18373,1.18394,1.183860,201.0,ask,True
1,2021-08-05 00:00:00+00:00,1.18367,1.18380,1.18358,1.18380,1.183712,201.0,bid,True
2,2021-08-05 00:15:00+00:00,1.18393,1.18395,1.18368,1.18369,1.183813,235.0,ask,True
3,2021-08-05 00:15:00+00:00,1.18379,1.18381,1.18353,1.18354,1.183667,235.0,bid,True
4,2021-08-05 00:30:00+00:00,1.18370,1.18373,1.18345,1.18357,1.183612,200.0,ask,True
...,...,...,...,...,...,...,...,...,...
395,2021-08-09 01:15:00+00:00,1.17518,1.17545,1.17513,1.17541,1.175293,340.0,bid,True
396,2021-08-09 01:30:00+00:00,1.17554,1.17594,1.17534,1.17586,1.175670,490.0,ask,True
397,2021-08-09 01:30:00+00:00,1.17539,1.17579,1.17518,1.17571,1.175517,490.0,bid,True
398,2021-08-09 01:45:00+00:00,1.17588,1.17590,1.17560,1.17562,1.175750,245.0,ask,False


In [21]:
latest_candle = candles_generator.get_latest_candle()

INFO:imprint_trading_bot:mode: on_change change of extremes for EUR_USD, M15


Candle completed: {"close_time":{"0":1628474400000,"1":1628474400000},"open":{"0":1.17568,"1":1.17553},"high":{"0":1.17568,"1":1.175605},"low":{"0":1.175605,"1":1.17553},"close":{"0":1.175605,"1":1.175605},"ohlc4":{"0":1.1756425,"1":1.1755675},"volume":{"0":2.0,"1":2.0},"price":{"0":"ask","1":"bid"},"complete":{"0":false,"1":false}}


In [22]:
latest_candle['kline']

,close_time,open,high,low,close,ohlc4,volume,price,complete
0,2021-08-09 02:00:00,1.17568,1.175680,1.175605,1.175605,1.175642,2.0,ask,False
1,2021-08-09 02:00:00,1.17553,1.175605,1.175530,1.175605,1.175568,2.0,bid,False


In [8]:
latest_candle['kline'].query('price == "ask"').close_time

0   2021-08-06 20:45:00+00:00
Name: close_time, dtype: datetime64[ns, UTC]

In [9]:
latest_candle['kline'].query('price == "ask"').close_time.dt.weekday

0    4
Name: close_time, dtype: int64

In [29]:
if -1:
    print("A")

A


In [33]:
import datetime
import calendar
import re

def granularity_to_time(s):
    """
    get value in seconds for named granularities: M1, M5 ... H1 etc.
    """
    mfact = {
        'S': 1,
        'M': 60,
        'H': 3600,
        'D': 86400,
    }
    try:
        f, n = re.match("(?P<f>[SMHD])(?:(?P<n>\d+)|)", s).groups()
        n = n if n else 1
        return mfact[f] * int(n)
    except AttributeError:
        raise UnknownGranularity(s)

        
time = datetime.datetime.utcnow() 
dt = pd.to_datetime(time)
epoch = int(calendar.timegm(dt.timetuple()))
        
frame_time = granularity_to_time("M15")

start = epoch - (epoch % frame_time)
end = epoch - (epoch % frame_time) + frame_time
print(start, end)

1628736300 1628737200


60.0

# Manual Stuff

In [31]:
PRACTICE = True

ctx = API(
    environment='practice' if PRACTICE else 'live',
    access_token=DEMO_TOKEN if PRACTICE else TOKEN,
    )

In [5]:
params = {
    #'granularity': "M30",
    'granularity': "M15",
    'count': BATCH_SIZE,
    'alignmentTimezone': "UTC",
    'price': "AB"
}

candles_req = instruments.InstrumentsCandles(instrument="EUR_USD", params=params)
response = ctx.request(candles_req)
# candles_req.response

Su Binance usi ask

In [6]:
klines_df = pd.DataFrame(sum([process_oanda_candle(candle) 
                              for candle in candles_req.response.get("candles", 200)], []))
klines_df.tail()

,close_time,open,high,low,close,ohlc4,volume,price,complete
395,2021-08-02 01:00:00+00:00,1.18685,1.18692,1.18645,1.18681,1.186758,406.0,bid,True
396,2021-08-02 01:15:00+00:00,1.18695,1.18697,1.18669,1.18670,1.186828,271.0,ask,True
397,2021-08-02 01:15:00+00:00,1.18679,1.18683,1.18655,1.18657,1.186685,271.0,bid,True
398,2021-08-02 01:30:00+00:00,1.18669,1.18669,1.18630,1.18650,1.186545,447.0,ask,False
399,2021-08-02 01:30:00+00:00,1.18653,1.18653,1.18616,1.18637,1.186397,447.0,bid,False


In [7]:
klines_df.complete.all()

False

In [12]:
import oandapyV20
import oandapyV20.endpoints.instruments as instruments
client = oandapyV20.API(access_token=DEMO_TOKEN, environment='practice')
params = {
    #'granularity': "M30",
    'granularity': "M15",
    'count': 2,
    'alignmentTimezone': "UTC",
    'price': "AB"
}

r = instruments.InstrumentsCandles(instrument="EUR_USD", params=params)
client.request(r)
print( r.response)


{'instrument': 'EUR_USD', 'granularity': 'M15', 'candles': [{'complete': True, 'volume': 332, 'time': '2021-08-06T20:30:00.000000000Z', 'bid': {'o': '1.17587', 'h': '1.17638', 'l': '1.17578', 'c': '1.17618'}, 'ask': {'o': '1.17602', 'h': '1.17655', 'l': '1.17592', 'c': '1.17634'}}, {'complete': True, 'volume': 312, 'time': '2021-08-06T20:45:00.000000000Z', 'bid': {'o': '1.17616', 'h': '1.17620', 'l': '1.17590', 'c': '1.17605'}, 'ask': {'o': '1.17632', 'h': '1.17638', 'l': '1.17613', 'c': '1.17626'}}]}


## Streaming Candles

In [26]:
from generators.candlefactory import CandleFactory, StreamRecord
from oandapyV20.endpoints.pricing import PricingStream

candle_factory = CandleFactory(
    instrument='EUR_USD', 
    granularity='M15', 
    processing_mode='on_end')


pricing_stream = PricingStream(
    accountID=oanda_demo_headers['id'],
    params={"instruments": "EUR_USD", 'alignmentTimezone': "UTC"}
)

PRACTICE = True

ctx = API(
    environment='practice' if PRACTICE else 'live',
    access_token=DEMO_TOKEN,
    )

INFO:imprint_trading_bot:processing mode: on_end


In [ ]:
# from oandapyV20.exceptions import V20Error, StreamTerminated

TICK = 1
HEARTBEAT = 2

while True:
    for rv in ctx.request(pricing_stream):
        s = StreamRecord(rv)
        candle = candle_factory.process_tick(s)
        if candle:
            print("Candle completed: {}".format(json.dumps(candle)))
        if s.recordtype() == TICK:
            print(s)


# Historical

In [36]:
from oandapyV20.contrib.factories import InstrumentsCandlesFactory


instrument = "EUR_USD"
params = {
    "from": "2021-08-01T00:00:00Z",
    "granularity": "M15",
    'alignmentTimezone': "UTC",
    'price': "AB"
}

klines_df = pd.DataFrame()
for r in InstrumentsCandlesFactory(instrument=instrument, params=params):
    rv = ctx.request(r)
    batch = r.response
    # r.response
    klines_df = klines_df.append(
        pd.DataFrame(
            sum([process_oanda_candle(candle) for candle in batch.get("candles", 200)], [])))


,close_time,open,high,low,close,ohlc4,volume,price,complete
0,2021-08-01 21:00:00+00:00,1.18673,1.18752,1.18673,1.18734,1.187080,14.0,ask,True
1,2021-08-01 21:00:00+00:00,1.18573,1.18652,1.18573,1.18634,1.186080,14.0,bid,True
2,2021-08-01 21:30:00+00:00,1.18720,1.18727,1.18699,1.18727,1.187183,12.0,ask,True
3,2021-08-01 21:30:00+00:00,1.18637,1.18639,1.18633,1.18639,1.186370,12.0,bid,True
4,2021-08-01 21:45:00+00:00,1.18718,1.18729,1.18689,1.18689,1.187062,21.0,ask,True
...,...,...,...,...,...,...,...,...,...
123,2021-08-06 20:15:00+00:00,1.17590,1.17611,1.17585,1.17587,1.175933,185.0,bid,True
124,2021-08-06 20:30:00+00:00,1.17602,1.17655,1.17592,1.17634,1.176208,332.0,ask,True
125,2021-08-06 20:30:00+00:00,1.17587,1.17638,1.17578,1.17618,1.176052,332.0,bid,True
126,2021-08-06 20:45:00+00:00,1.17632,1.17638,1.17613,1.17626,1.176273,312.0,ask,True
